In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the visits data
visit_data_path = ('/mnt/data/visits_log_us.csv') 
visits_data = pd.read_csv(visits_data_path)

In [ ]:
# Display the first few rows of the dataset
visits_data.head()

In [ ]:
# convert 'Start Ts' and 'End Ts' to datetime format
visits_data['Start Ts'] = pd.to_datetime(visits_data['Start Ts'])
visits_data['End Ts'] = pd.to_datetime(visits_data['End Ts'])

In [ ]:
# Calculate the session duration in minutes 
visits_data['Session Duration'] = (visits_data['End Ts'] - visits_data['Start Ts']).dt.total_seconds() / 60

In [ ]:
#check for missing values and duplicates
missing_values = visits_data.isnull() .sum()
duplicates = visits_data.duplicated() .sum()

In [ ]:
#Load the orders data
orders_data_path = '/mnt/data/orders_log.us.csv'
orders_data = pd.read_csv(orders_data_path)

In [ ]:
# Display the first few rows of the dataset
orders_data.head()

In [ ]:
# Convert 'Buy Ts' to datetime format
orders_data['Buy Ts'] = pd.to_datetime(orders_data['Buy Ts'])

In [ ]:
# Check for missing values and duplicates
missing_values_orders = orders_data.isnull().sum()

In [ ]:
# Check for missing values and duplicates
missing_values_orders = orders_data.isnull().sum()
duplicates_orders = orders_data.duplicated().sum()

In [ ]:
# Load the costs data
costs_data_path = '/mnt/data/costs_us.csv'
costs_data = pd.read_csv(costs_data_path)

In [ ]:
# Convert 'dt' to date format
costs_data['dt'] = pd.to_datetime(costs_data['dt'])

In [ ]:
# Check for missing values and duplicates
missing_values_costs = costs_data.isnull().sum()
duplicates_costs = costs_data.duplicated().sum()

In [ ]:
# Calculate Daily Active Users (DAU), Weekly Active Users (WAU), Monthly Active Users (MAU)
visits_data['day'] = visits_data['Start Ts'].dt.date
visits_data['week'] = visits_data['Start Ts'].dt.isocalendar().week
visits_data['month'] = visits_data['Start Ts'].dt.to_period('M')

In [ ]:
# DAU:
visits_data['day'] = visits_data['Start Ts'].dt.date
DAU = visits_data.groupby('day')
['Uid'].nunique()

In [ ]:
DAU.plot(figsize=(10, 6))
plt.title('Daily Active User')
plt.xlabel('Day')
plt.ylabel('Number of User')
plt.show()

In [ ]:
# WAU:
visits_data['week'] = visits_data['Start Ts'].dt.strftime('%Y-%W')
WAU = visits_data.groupby('week')
['Uid'].nunique()

In [ ]:
WAU.plot(figsize=(10, 6))
plt.title('Weekly Active Users')
plt.xlabel('Week')
plt.ylable('Number of Users')
plt.show()

In [ ]:
# MAU:
visits_data['month'] = visits_data['Start Ts'].dt.strftime('%Y-%M')
MAU = visits_data.groupby('month')
['Uid'].nunique()

In [ ]:
MAU.plot(figsize=(10, 6))
plt.title('Monthly Active Users')
plt.xlabel('Month')
plt.ylabel('Number of Users')
plt.show()

In [ ]:
#Calculate Number of Sessions per Day
sessions_per_day = visits_data.groupby('day').size()

In [ ]:
#Number of Sessions per Day
sessions_per_day.plot(figsize=(10, 6))
plt.title('Number of Sessions per Day')
plt.xlabel('Day')
plt.ylabel('Number of Sessions')
plt.show()

In [ ]:
#Calculate Average Session Lenth
avg_session_length = visits_data['Session Duration'].mean()

In [ ]:
#Assuming avg_session_length is a single value
plt.figure(figsize=(5, 5))
sns.barplot(x=['Average Session Length'], y=[avg_session_length])
plt.ylabel('Minutes')
plt.show()

In [ ]:
# User Retention Rate
# Determined the first vist for each user

first_visit = visits_data.groupby('Uid')['Start Ts'].min()
visits_data = visits_data.merge(first_visit.reset_index(), on='Uid', how='left', suffixes=('', '_first'))

Sales Analysis

In [ ]:
# Merge the first visit date with the orders data:
orders_data = orders_data.merge(first_visit.reset_index(), on='Uid', how='left')
orders_data['time_to_purchase'] = (orders_data['Buy Ts'] - orders_data['Start Ts_first']).dt.total_seconds() / 86400

In [ ]:
sns.histplot(orders_data['time_to_purchase'], bins=30, kde=False)
plt.title('Time to First Purchase Distribution')
plt.xlabel('Days')
plt.ylabel('Number of Users')
plt.show()

In [ ]:
# Number of Orders
#Grouped by the desired time period and count orders:

orders_per_period = orders_data.groupby(['period'])
['Uid'].count()

In [ ]:
#Number of Orders Over Time
orders_per_period.plot(figsize=(10, 6))
plt.title('Number of Orders Over Time')
plt.xlabel('Time Period')
plt.ylabel('Number of Orders')
plt.show()

Marketing Analysis

In [ ]:
# Total Expenditure

total_costs = costs_data['costs'].sum()

In [ ]:
#Costs Over Time
costs_data.groupby('dt')
['cost'].sum().plot(figsize=(10, 6))
plt.title('Marketing Costs Over Time')
plt.xlabel('Date')
plt.ylabel('Costs')
plt.show()
